#Imports


In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')


In [ ]:

# System and file operations
import os
import json

# Numerical and data handling
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Machine learning metrics
from sklearn.metrics import confusion_matrix

# TensorFlow and Keras for deep learning
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    BatchNormalization, GlobalAveragePooling2D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2


Mounted at /content/drive


#Data

In [ ]:
def load_dataset(char_dir, img_size=(224, 224), batch_size=32):
    """
    Load dataset from the specified directory and create data generators for training, validation, and testing.

    Parameters:
    char_dir (str): Directory containing the dataset.
    img_size (tuple): Target size for resizing images.
    batch_size (int): Number of samples per batch.

    Returns:
    tuple: Training, validation, and testing data generators.
    """
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    train_generator = datagen.flow_from_directory(
        char_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    val_generator = datagen.flow_from_directory(
        char_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=True
    )

    test_datagen = ImageDataGenerator(rescale=1./255)

    test_generator = test_datagen.flow_from_directory(
        char_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, val_generator, test_generator


In [ ]:
# Load Data

main_dir = '/content/drive/MyDrive/IndividualShapeDataset1505'

# Load the Fill dataset
fill_char_dir = os.path.join(main_dir, 'Fill')
fill_train_generator, fill_val_generator, fill_test_generator = load_dataset(fill_char_dir)

# Load the Shape dataset
shape_char_dir = os.path.join(main_dir, 'Shape')
shape_train_generator, shape_val_generator, shape_test_generator = load_dataset(shape_char_dir)



Found 1105 images belonging to 3 classes.
Found 275 images belonging to 3 classes.
Found 1380 images belonging to 3 classes.
Found 1105 images belonging to 3 classes.
Found 275 images belonging to 3 classes.
Found 1380 images belonging to 3 classes.
Found 1091 images belonging to 3 classes.
Found 271 images belonging to 3 classes.
Found 1362 images belonging to 3 classes.


#Saving

In [ ]:
# Save functions

def save_history(history, model_name):
    """
    Save the training history to a JSON file.

    Parameters:
    history (History): Training history.
    model_name (str): Name of the model.
    """
    history_dict = history.history
    for key in history_dict:
        history_dict[key] = [float(x) for x in history_dict[key]]
    history_path = f'/content/drive/MyDrive/Models/Characteristics/17052024/{model_name}_history.json'
    with open(history_path, 'w') as f:
        json.dump(history_dict, f)
    print(f"History saved at {history_path}")

def save_model_info(model, model_name, test_acc):
    """
    Save model information to a JSON file.

    Parameters:
    model (Model): Trained model.
    model_name (str): Name of the model.
    test_acc (float): Test accuracy of the model.
    """
    model_info = {
        'model_name': model_name,
        'test_accuracy': test_acc,
        'input_shape': model.input_shape[1:],
        'num_layers': len(model.layers),
        'num_classes': model.output_shape[-1]
    }
    model_info_path = f'/content/drive/MyDrive/Models/Characteristics/17052024/{model_name}_info.json'
    with open(model_info_path, 'w') as f:
        json.dump(model_info, f)
    print(f"Model info saved at {model_info_path}")

def save_model(model, model_name, history, fine_tune_history, test_acc):
    """
    Save the trained model and its related information.

    Parameters:
    model (Model): Trained model.
    model_name (str): Name of the model.
    history (History): Training history.
    fine_tune_history (History): Fine-tuning history.
    test_acc (float): Test accuracy of the model.
    """
    model_path = f'/content/drive/MyDrive/Models/Characteristics/17052024/{model_name}_model.h5'
    model.save(model_path)
    print(f"Model saved at {model_path}")
    save_history(history, model_name)
    save_history(fine_tune_history, f'{model_name}_fine_tuned')
    save_model_info(model, model_name, test_acc)


#Model

In [ ]:
# Model creation
def create_model(base_model, num_classes, input_shape=(224, 224, 3)):
    """
    Create a new model based on a pre-trained base model.

    Parameters:
    base_model (Model): Pre-trained base model.
    num_classes (int): Number of output classes.
    input_shape (tuple): Input shape of the model.

    Returns:
    Model: Compiled model.
    """
    base_model = base_model(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

def train_model(model, train_generator, val_generator, test_generator, epochs, batch_size, model_name):
    """
    Train the model with the provided data generators and save the model.

    Parameters:
    model (Model): Model to be trained.
    train_generator (DirectoryIterator): Training data generator.
    val_generator (DirectoryIterator): Validation data generator.
    test_generator (DirectoryIterator): Testing data generator.
    epochs (int): Number of epochs to train the model.
    batch_size (int): Batch size for training.
    model_name (str): Name of the model.

    Returns:
    Model: Trained model.
    History: Training history.
    History: Fine-tuning history.
    float: Test accuracy.
    """
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'best_{model_name}_model.keras', save_best_only=True, monitor='val_loss')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)

    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size,
        epochs=epochs,
        callbacks=[early_stopping, model_checkpoint, reduce_lr]
    )

    val_loss, val_acc = model.evaluate(val_generator, steps=val_generator.samples // batch_size)
    print(f"Validation accuracy for {model_name}: {val_acc:.4f}")

    for layer in model.layers[-5:]:
        layer.trainable = True

    model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

    fine_tune_history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size,
        epochs=10,  # Fine-tuning epochs
        callbacks=[early_stopping, model_checkpoint, reduce_lr]
    )

    test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
    print(f"Test accuracy for {model_name}: {test_acc:.4f}")

    save_model(model, model_name, history, fine_tune_history, test_acc)

    return model, history, fine_tune_history, test_acc


#Training

In [ ]:
# Set the number of epochs, batch size and input Dimensions
epochs = 50
batch_size = 32
input_shape = (224, 224, 3)

In [ ]:
# Train and save the Fill model
fill_num_classes = len(fill_train_generator.class_indices)
fill_model = create_model(MobileNetV2, fill_num_classes, input_shape)
fill_model, fill_history, fill_fine_tune_history, fill_test_acc = train_model(
    fill_model,
    fill_train_generator,
    fill_val_generator,
    fill_test_generator,
    epochs,
    batch_size,
    'fill_model'
)
save_model(fill_model, 'fill_model', fill_history, fill_fine_tune_history, fill_test_acc


9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/50
34/34 [==============================] - 81s 2s/step - loss: 0.4227 - accuracy: 0.8444 - val_loss: 0.0957 - val_accuracy: 0.9844 - lr: 0.0010
Epoch 2/50
34/34 [==============================] - 79s 2s/step - loss: 0.1400 - accuracy: 0.9646 - val_loss: 0.0531 - val_accuracy: 0.9844 - lr: 0.0010
Epoch 3/50
34/34 [==============================] - 77s 2s/step - loss: 0.0977 - accuracy: 0.9711 - val_loss: 0.0872 - val_accuracy: 0.9844 - lr: 0.0010
Epoch 4/50
34/34 [==============================] - 71s 2s/step - loss: 0.0868 - accuracy: 0.9795 - val_loss: 0.0637 - val_accuracy: 0.9844 - lr: 0.0010
Epoch 5/50
34/34 [==============================] - 78s 2s/step - loss: 0.0549 - accuracy: 0.9795 - val_loss: 0.1050 - val_accuracy: 0.9844 - lr: 0.0010
Epoch 6/50
34/34 [==============================] - 79s 2s/step - loss: 0.0518 - accuracy: 0.9879 - val_loss: 0.0686 - val_accuracy: 0.9844 - lr: 1.0000e-04
Epoch 7/50
8/8 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved at /content/drive/MyDrive/Models/Characteristics/17052024/fill_model_model.h5
History saved at /content/drive/MyDrive/Models/Characteristics/17052024/fill_model_history.json
History saved at /content/drive/MyDrive/Models/Characteristics/17052024/fill_model_fine_tuned_history.json
Model info saved at /content/drive/MyDrive/Models/Characteristics/17052024/fill_model_info.json
Model saved at /content/drive/MyDrive/Models/Characteristics/17052024/fill_model_model.h5
History saved at /content/drive/MyDrive/Models/Characteristics/17052024/fill_model_history.json
History saved at /content/drive/MyDrive/Models/Characteristics/17052024/fill_model_fine_tuned_history.json
Model info saved at /content/drive/MyDrive/Models/Characteristics/17052024/fill_model_info.json


In [ ]:
# Train and save the Shape model
shape_num_classes = len(shape_train_generator.class_indices)
shape_model = create_model(MobileNetV2, shape_num_classes, input_shape)
shape_model, shape_history, shape_fine_tune_history, shape_test_acc = train_model(
    shape_model,
    shape_train_generator,
    shape_val_generator,
    shape_test_generator,
    epochs,
    batch_size,
    'shape_model'
)
save_model(shape_model, 'shape_model', shape_history, shape_fine_tune_history, shape_test_acc)


Epoch 1/50
34/34 [==============================] - 379s 11s/step - loss: 0.3448 - accuracy: 0.8714 - val_loss: 0.0220 - val_accuracy: 0.9961 - lr: 0.0010
Epoch 2/50
34/34 [==============================] - 79s 2s/step - loss: 0.0692 - accuracy: 0.9832 - val_loss: 0.0055 - val_accuracy: 0.9961 - lr: 0.0010
Epoch 3/50
34/34 [==============================] - 76s 2s/step - loss: 0.0213 - accuracy: 0.9925 - val_loss: 0.0719 - val_accuracy: 0.9844 - lr: 0.0010
Epoch 4/50
34/34 [==============================] - 78s 2s/step - loss: 0.0255 - accuracy: 0.9944 - val_loss: 4.1445e-04 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/50
34/34 [==============================] - 71s 2s/step - loss: 0.0194 - accuracy: 0.9944 - val_loss: 0.0748 - val_accuracy: 0.9609 - lr: 0.0010
Epoch 6/50
34/34 [==============================] - 78s 2s/step - loss: 0.0050 - accuracy: 0.9991 - val_loss: 0.0011 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 7/50
34/34 [==============================] - 72s 2s/step - loss: 0.00